### Tasking completion notebook
This script is meant to be used on tasking orders in status "BEING FULFILLED" with a "Polygon" geometry input type. It is for repetitive tasking orders which can not leverage the existing API Get coverage of order assets end point.

The logic:

 - Find active tasking orders in your account (can fine tune to a specific workspace if needed) using 'Tags'
 - Extract the order id, the input AOI geometry and compute in the input AOI surface area
 - Use the order id to find delivered assets
 - Compute delivered assets coverage area in km2, compute coverage percentage.

### Libraries

In [154]:
import up42
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from shapely.ops import unary_union

### Learn - step by step

In [ ]:
up42.authenticate('../../../central_creds/credentials.json')
storage = up42.initialize_storage()

#### Find active tasking orders


Extract geometry and order id from active order using tags.

In [ ]:
# my active orders

orders = storage.get_orders(workspace_orders=False,
                            order_type='TASKING',
                            statuses=['BEING_FULFILLED'],
                            limit=5,
                            tags=['cs_tag'])

In [ ]:
order_ids_list = [order.order_id for order in orders]
order_ids_list

In [219]:
order_info = orders[0].info
# order_info

In [220]:
tasking_geometry = orders[0].order_details['geometry']
# tasking_geometry

# Create the Polygon from coordinates
polygon = Polygon(tasking_geometry['coordinates'][0])


In [ ]:
order_dataframe = gpd.GeoDataFrame([[order_info['id'], order_info['status'], order_info['tags'], polygon]],
                 columns=['order_id', 'status', 'tags', 'geometry'],
                 crs='EPSG:4326')
order_dataframe

In [167]:
best_utm_zone = order_dataframe.estimate_utm_crs()
# best_utm_zone

In [170]:
order_dataframe_utm = order_dataframe.to_crs(best_utm_zone)
# order_dataframe_utm

In [ ]:
order_dataframe_utm_area = order_dataframe_utm.area/1000000
order_dataframe_utm['aoi_area_km'] = round(order_dataframe_utm_area,2)
order_dataframe_utm

#### Delivered data in storage

Find delivered data using the order id, acquisition dates and cloud cover

In [193]:
start_date = "2022-06-01"
end_date = "2023-06-15"
order_id = order_info['id']
cloud_cover = 50

In [194]:
pystac_client = storage.pystac_client

custom_filter = {
    "op":"and", "args":[
    {"op": "<","args": [{"property": "eo:cloud_cover"}, cloud_cover]},
    {"op": "=","args": [{"property": "order_id"},order_id]},
    {"op": "t_overlaps","args": [{"property": "datetime"},{"interval": [start_date, end_date]}]}
  ]
}

assets_search = pystac_client.search(filter=custom_filter, filter_lang='cql2-json')


In [195]:
assets = assets_search.get_all_items()

In [ ]:
delivered_assets_geom_list = [Polygon(asset.geometry['coordinates'][0]) for asset in assets]
delivered_assets_geom_list

In [ ]:
merged_delivered_assets_geoms = unary_union(delivered_assets_geom_list)
merged_delivered_assets_geoms

In [ ]:
delivered_assets_dt = gpd.GeoDataFrame(geometry=[merged_delivered_assets_geoms], crs='EPSG:4326')
delivered_assets_dt.estimate_utm_crs()
delivered_assets_dt_utm = delivered_assets_dt.to_crs(best_utm_zone)
delivered_assets_dt_utm

In [ ]:
order_summary = order_dataframe_utm
order_summary['delivered_area_km'] = round(delivered_assets_dt_utm.area/1000000, 2)
order_summary['delivered_%'] = round(order_summary['delivered_area_km']*100/order_summary['aoi_area_km'], 2)
order_summary

#### Automated

In [ ]:
up42.authenticate('../../../central_creds/credentials.json')
storage = up42.initialize_storage()

# Search filter parameters - define these as needed
start_date = "2022-06-01"
end_date = "2023-06-15"
cloud_cover = 50

# Find orders - modify or remove the tags as needed
orders = storage.get_orders(workspace_orders=False,
                            order_type='TASKING',
                            statuses=['BEING_FULFILLED'],
                            limit=50,
                            tags=['pastis', 'cs_tag'])


# Build order id list
order_ids_list = [order.order_id for order in orders]

all_delivered_assets_extents = gpd.GeoDataFrame()
all_order_summaries = gpd.GeoDataFrame()

for num, order_id in enumerate(order_ids_list):

    print(f'Working on order {order_id}')

    # Extract order information
    order_info = orders[num].info

    # Etract the AOI for this order
    tasking_geometry = orders[num].order_details['geometry']

    # Create the Polygon from coordinates
    polygon = Polygon(tasking_geometry['coordinates'][0])

    # Create dataframe for the order
    order_dataframe = gpd.GeoDataFrame([[order_info['id'], order_info['status'], order_info['tags'], polygon]],
                    columns=['order_id', 'status', 'tags', 'geometry'],
                    crs='EPSG:4326')

    # Estimate the best UTM zone
    best_utm_zone = order_dataframe.estimate_utm_crs()

    # Convert dataframe to UTM
    order_dataframe_utm = order_dataframe.to_crs(best_utm_zone)

    # Compute the tasking aoi area size
    order_dataframe_utm_area = order_dataframe_utm.area/1000000
    order_dataframe_utm['aoi_area_km'] = round(order_dataframe_utm_area,2)

    # Find delivered data/assets in storage that correspond to this order
    print(' - Searching assets for order...')
    # assets = storage.get_assets(search=order_id)

    pystac_client = storage.pystac_client
    custom_filter = {
        "op":"and", "args":[
        {"op": "<","args": [{"property": "eo:cloud_cover"}, cloud_cover]},
        {"op": "=","args": [{"property": "order_id"},order_id]},
        {"op": "t_overlaps","args": [{"property": "datetime"},{"interval": [start_date, end_date]}]}
        ]
    }
    assets_search = pystac_client.search(filter=custom_filter, filter_lang='cql2-json')
    assets = assets_search.get_all_items()
    
    if len(assets) > 0:

        # Place the assets geometries into a list and merge then together
        delivered_assets_geom_list = [Polygon(asset.geometry['coordinates'][0]) for asset in assets]
        merged_delivered_assets_geoms = unary_union(delivered_assets_geom_list)

        # Create delivered assets dataframe and convert to UTM
        delivered_assets_dt = gpd.GeoDataFrame(geometry=[merged_delivered_assets_geoms], crs='EPSG:4326')
        delivered_assets_dt.estimate_utm_crs()
        delivered_assets_dt_utm = delivered_assets_dt.to_crs(best_utm_zone)

        all_delivered_assets_extents = pd.concat([all_delivered_assets_extents,delivered_assets_dt])

        # Create an order summary dataframe with coverage percentage for the order
        order_summary = order_dataframe_utm
        order_summary['delivered_area_km'] = round(delivered_assets_dt_utm.area/1000000, 2)
        order_summary['delivered_%'] = round(order_summary['delivered_area_km']*100/order_summary['aoi_area_km'], 2)
        completion_percentage = order_summary['delivered_%'].values[0]
        print(f' - {completion_percentage}% complete')

        all_order_summaries = pd.concat([all_order_summaries, order_summary.to_crs('EPSG:4326')], ignore_index=True)
    
    else:
        print(f' - No assets delivered yet for order: {order_id}')
        continue

all_order_summaries

In [ ]:
import folium

m = folium.Map(location=(0, 0), zoom_start = 2)
folium.GeoJson(data=all_order_summaries).add_to(m)
folium.GeoJson(data=all_delivered_assets_extents, color='Red').add_to(m)
m


In [ ]:
order_dataframe